# Adding Post-Newtonian general relativity corrections

It's easy to add post-newtonian corrections to your REBOUND simulations with REBOUNDx.  Let's start with a simulation without GR:

In [11]:
import rebound
sim = rebound.Simulation()
sim.add(m=1.) # Sun
sim.add(m=1.66013e-07,a=0.387098,e=0.205630) # Mercury-like
sim.move_to_com() # Moves to the center of momentum frame

sim.integrate(10.)
print("pomega = %.16f"%sim.particles[1].pomega)

pomega = 0.0000000000000000


As expected, the pericenter did not move at all.  Now let's add GR

In [12]:
import reboundx
rebx = reboundx.Extras(sim)
params = rebx.add_gr()

and run for another 100 time units

In [13]:
deltat = 100.
sim.integrate(sim.t + deltat)
print("pomega = %.16f"%sim.particles[1].pomega)
juliancentury = 628.33195 # in yr/2pi
arcsec = 4.8481368e-06 # in rad
print("Rate of change of pomega = %.4f [arcsec / Julian century]"% (sim.particles[1].pomega/deltat*juliancentury/arcsec))

pomega = 0.0000000000000000
Rate of change of pomega = 0.0000 [arcsec / Julian century]


As expected there was pericenter precession. The literature value is 42.98 arcsec / century. 

**Units**

To add GR, you need to pass `add_gr` the speed of light `c` in the units appropriate to the simulation.  By not passing a value above, `c` defaulted to our default units of AU, $M_\odot$ and yr/$2\pi$, for which `G=1` which is what we used for our initial conditions above.

But imagine now we wanted to instead use SI units:

In [14]:
sim = rebound.Simulation()
sim.G = 6.67408e-11
sim.add(m=1.989e30) # Sun
sim.add(m=3.30216458e23,a=5.79090366e10,e=0.205630) # SI units
sim.move_to_com() # Moves to the center of momentum frame
rebx = reboundx.Extras(sim)

If you call `add_gr()` like before you'll get an error telling complaining you changed `G`.  You must explicitly pass `c`:

In [15]:
params = rebx.add_gr(c=3.e8)

Currently, setting the `units` member in Simulation does not work with REBOUNDx.  If you want to use your own units you always have to pass `c` explicitly.

**Variants**

Above we always called `add_gr`, but there are other options depending on your application:

* `add_gr` (default): Assumes that only a single mass is the source of post-newtonian corrections (Eq. 2 of Benitez and Gallardo 2008).  This should be good enough for most applications with planets orbiting stars in which GR matters.  Gets both the mean motion and precession correct.  Will be significantly faster than `add_gr_full`, particularly with several bodies.

* `add_gr_potential`: This is the simplest potential you can use for GR.  It gets the precession right, but the mean motion is wrong by $\mathcal{O}(GM/ac^2)$.  Like `add_gr`, it takes a single particle to be the source of the GR effects.  It is the fastest option, and because it's not velocity-dependent, it automatically keeps WHFAST symplectic.  Nice if you don't need to get GR exactly right, and want speed.

* `add_gr_full`: Allows for multiple massive bodies (Eq. 1 of Benitez and Gallardo 2008).  This gets the GR corrections to the mean motion and the precession correct, and it doesn't matter how you order your planets.  This is the safe option, though it is slower.

To set the source particle for `add_gr` or `add_gr_potential`, you'd pass the appropriate particle, e.g., `add_gr(source=sim.particles[2])`.
